In [10]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv

In [26]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv')
weibo_o = pd.read_csv('./data/Weibo.csv')
wechat_o = pd.read_csv('./data/Wechat.csv')
news_forum_o = pd.read_csv('./data/News_forum.csv')

In [100]:
weibo_m = weibo_m_o[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
weibo = weibo_o[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
wechat = wechat_o[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
news_forum = news_forum_o[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')

In [101]:
weibo_m.columns = ['CONTENT','AREAS AFFECTED']
weibo.columns = ['CONTENT','AREAS AFFECTED']

In [102]:
data = pd.concat([weibo_m,weibo,wechat,news_forum])

In [103]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('life', 'Life')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('sponsored', 'Sponsored')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('news', 'News')
data = data.reset_index(drop=True)

In [104]:
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')

In [105]:
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('News', 'Brands')
for i in range(len(data)):
    if ((data['AREAS AFFECTED'][i] != 'Products/Service') and 
        (data['AREAS AFFECTED'][i] != 'Corporate Brands')):
        data['AREAS AFFECTED'][i] = 'others'

In [106]:
mypath = './data/processed/'
if not os.path.exists(mypath):
    os.makedirs(mypath)
for the_file in os.listdir(mypath):
    file_path = os.path.join(mypath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [107]:
for x in data['AREAS AFFECTED'].unique():
    np.savetxt(mypath+ x.replace('/',' ') +'.utf8',\
                   data[data['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')

In [108]:
data['AREAS AFFECTED'].value_counts()

others              15095
Products/Service     9062
Corporate Brands     6737
Name: AREAS AFFECTED, dtype: int64

# Sentiment data

In [27]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [28]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [29]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [30]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [31]:
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')

In [32]:
mypath = './data/processed/'
if not os.path.exists(mypath):
    os.makedirs(mypath)
for the_file in os.listdir(mypath):
    file_path = os.path.join(mypath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [33]:
for x in data['SENTIMENT'].unique():
    np.savetxt(mypath+ x.replace('/',' ') +'.utf8',\
                   data[data['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

In [34]:
data['SENTIMENT'].value_counts()

Neutral       14618
Positive        465
Negative        287
Irrelevant      253
Name: SENTIMENT, dtype: int64

In [39]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model.bak')

In [40]:
vocab_obj = w2vModel.wv.vocab
len(vocab_obj)

3911

In [13]:
from train import *
x_text, y = data_helpers.load_positive_negative_data_files(FLAGS)

ArgumentError: argument --dev_sample_percentage: conflicting option string(s): --dev_sample_percentage

In [14]:
from train import *

ArgumentError: argument --dev_sample_percentage: conflicting option string(s): --dev_sample_percentage